# MiniProject 1: Storage capacity in biologically plausible Hopfield networks #

## Introduction ##

The Hopfield model is a standard model in computational neuroscience that models the storage of memory items, in the form of “patterns” of neuronal activity, in the recurrent connectivity of a neural network.

The aim of this project is to investigate the robustness of memory retrieval in Hopfield networks with biologically plausible constraints. The lectures mostly covered standard Hopfield networks with balanced patterns and a symmetric weight matrix.

However, in biological networks, neural activity is generally sparse with only a few neurons active at a time.

Moreover, the symmetric connectivity of the standard Hopfield model is inconsistent with Dale’s law which states that the outgoing synapses from each neuron should be either excitatory or inhibitory; and it is very unlikely to find symmetric connectivity in the brain.

To address these issues, we will generalise the Hopfield model to low-activity patterns and separated excitatory and inhibitory populations. We will first start with a classic symmetric Hopfield network, and investigate the capacity of this network in storing balanced random patterns, i.e. with 50% of active neurons in the network.

In the second part, we will simulate a network with low-activity patterns. Finally, in the third section, we will separate the network into excitatory and inhibitory populations, and explore memory retrieval.

Note: the project is intended to be solved using Python without the need for any specific library (other than the usual numpy and matplotlib). You are free to use other libraries if you want

## Ex 0. Getting Started: Standard Hopfield Network ##

To get started, we first consider the classical Hopfield model with balanced random patterns, consisting of $N$ fully connected, continuously-valued nodes $S_i(t) \in [−1,\, 1]$. The $M$ memory patterns $P^{\mu} \in \{−1, 1\}^{N}$ where each component is either +1 or −1 with probability $\frac{1}{2}$, are stored in the network by the weight matrix given in the standard Hebbian form.

$$
W_{ij} = \frac{1}{N} \sum_{\mu = 1}^M P^{\mu}_i P^{\mu}_j
$$

At each time step, the states update according to the rule:

$$
S_i(t + 1) = \phi \Biggl( \sum_{j = 1}^N W_{ij} S_j(t) \Biggr)
$$

where $\phi(h) = \tanh(\beta h)$, and we use $\beta = 4$

## Ex 0.1 ##

Write a method that generates binary balanced random patterns; and a method that computes the next state $S(t + 1)$ of the network, given the current state $S(t) = (S_1(t), . . . , S_N(t))$ and a set of patterns $P^1, ..., P^M$ according to eqs.(1)-(2).

In [27]:
%matplotlib inline
from neurodynex3.hopfield_network import network, pattern_tools, plot_tools
from matplotlib import pyplot as plt
import numpy as np

N = 100  # Number of neurons
M = 5    # Number of patterns
beta = 4 # The temperature

# Generate a random pattern:
def generate_random_pattern(N):
    P = np.random.binomial(1, 0.5, N)
    P = P * 2 - 1  # map {0, 1} to {-1 +1}
    return P

# Generate M random patterns:
def generate_pattern_set(M, N):
    P_set = np.zeros((M, N))
    for mu in range(M):
        P_set[mu] = generate_random_pattern(N)
    return P_set

# The phi function:
def phi(beta, h):
    return np.tanh(beta*h)

# Calculate the S(t+1) values:
def next_state(S_old, P_set):
    S_new = np.zeros(N)
    W = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            W[i, j] = np.sum(P_set[:, i] * P_set[:, j]) / N
    for j in range(N):
        S_new[j] = phi(beta, np.sum(W[:, j] * S_old[j]))
    return S_new

# Iterate n_step times:
def S_iterate(S_old, P_set, n_step):
    for i in range(n_step):
        S_old = next_state(S_old, P_set)
    return S_old

## Ex 0.2 ##

For a network with $N = 100$ neurons and $M = 5$ patterns, set the initial state close to the first pattern $P^1$.

To do this, randomly flip a given percentage c = 5% of neurons in the pattern.

Let the network evolve for 10-20 time steps until the network dynamics relax to a stable state.

Check the overlaps of the final state with all the patterns. Did the network correctly retrieve the first pattern?

In [37]:
%matplotlib inline
from neurodynex3.hopfield_network import network, pattern_tools, plot_tools

n_step = 20

# Pattern flipper:
def random_flip(P, c):
    P_new = np.copy(P)
    for i in range(int(c*N)):
        idx = np.random.randint(0, N)
        P_new[idx] = -P[idx]
    return P_new

P_set = generate_pattern_set(M, N)
S0 = random_flip(P_set[0], 0.05)
S_final = S_iterate(S0, P_set, n_step)

# Compute Overlap:
def overlap(P1, P2):
    return np.sum(P1*P2) / N

# Compute the overlap between the patterns and the final state:
overlap_list = np.zeros(M)
for mu in range(M):
    overlap_list[mu] = overlap(S_final, P_set[mu])
    print("Overlap between pattern %d and the final state: %.2f" % (mu, overlap_list[mu]))

Overlap between pattern 0 and the final state: 0.30
Overlap between pattern 1 and the final state: -0.04
Overlap between pattern 2 and the final state: -0.06
Overlap between pattern 3 and the final state: -0.13
Overlap between pattern 4 and the final state: 0.03
